In [ ]:
# COMPLETE BACKTEST (single cell) — All strategies S1..S4 with S3 = Aggressive/Scalp (C)
# Requirements: yfinance, pandas, numpy, matplotlib, tqdm
# pip install yfinance pandas numpy matplotlib tqdm

import os, json, time, re, math
from datetime import datetime, timedelta, timezone
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from pandas.api.types import is_scalar

# ---------------- USER CONFIG ----------------
SYMBOL = "ADANIPOWER.NS"    # ticker (change as needed)
INTERVAL = "1H"           # '1m','2m','5m','15m','30m','60m' or '1D'
PERIOD = "1Y"             # lookback
INITIAL_CASH = 100000.0

# ---------------- STRATEGY / ENGINE PARAMS ----------------
EMA_LEN = 200
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
ATR_LEN = 5                # ATR lookback

# Strategy sizing and rules for S1 stacking
S1_INIT_PCT = 0.80
S1_STACK_REL_TO_INITIAL = 0.3
S1_MAX_STACKS = 50
S1_STACK_DECAY = 0.90

# Option C flag for S1 exit logic (keeps your Option C behaviour)
USE_OPTION_C = True
S1_OPTION_C_PROFIT_PCT = 1.01

# S2 params
S2_PCT = 0.80
S2_TP_RR = 5.0
# S2 EPS change thresholds (Case A/B/C)
EPS_CHANGE_LONG_THRESH = 0.20  # +20% -> long only (Case A)
EPS_CHANGE_SHORT_THRESH = -0.20 # -20% -> short only (Case B)

# ---------------- S3 (Aggressive / Scalp C) ----------------
S3_PCT = 0.07            # 7% NAV (aggressive scalp)
S3_TP_RR = 1.5           # TP = entry - 1.5 * ATR (same)
S3_STOP_MULT = 0.4       # Stop = entry + 0.4 * ATR (tighter stop)
S3_MAX_HOLD_BARS = 24    # Max hold bars safety

# ---------------- S4 params (safer defaults) ----------------
S4_PCT = 0.15
S4_RR_TP = 2.0
S4_RR_STOP = 1.0
S4_MAX_CONCURRENT_PCT = 0.25
S4_MAX_OPEN_LOTS = 1
S4_COOLDOWN_BARS = 12

MIN_BARS_REQUIRED = 200

# ---------------- OUTPUT ----------------
OUT_DIR = "backtest_outputs_S3C"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------- Helpers: fetch price ----------------
def fetch_yf_intraday(symbol, period=PERIOD, interval=INTERVAL, out_dir=OUT_DIR):
    print(f"[yfinance] downloading {symbol} period={period} interval={interval} ...")
    df = yf.download(tickers=symbol, period=period, interval=interval, progress=False, threads=True, auto_adjust=False)
    if df is None or df.empty:
        raise RuntimeError("yfinance returned no price data. Check ticker/period/interval/network.")
    # flatten columns
    if isinstance(df.columns, pd.MultiIndex):
        try:
            df = df.xs(symbol, axis=1, level=0, drop_level=True)
        except Exception:
            df.columns = [col[-1] if isinstance(col, tuple) else col for col in df.columns.to_list()]
    else:
        df.columns = [c if isinstance(c, str) else str(c) for c in df.columns]

    required = ['Open','High','Low','Close','Volume']
    if not set(required).issubset(df.columns):
        if df.shape[1] >= 5:
            pos_names = ['Open','High','Low','Close','Adj Close','Volume']
            ncols = df.shape[1]
            inferred = pos_names[:ncols] if ncols <= len(pos_names) else [f"col{i}" for i in range(ncols)]
            df.columns = inferred
            if set(['Open','High','Low','Close']).issubset(df.columns):
                if 'Volume' not in df.columns:
                    last_col = df.columns[-1]
                    if last_col not in ['Open','High','Low','Close','Adj Close']:
                        df['Volume'] = pd.to_numeric(df[last_col], errors='coerce')
                    else:
                        df['Volume'] = np.nan
            else:
                raise RuntimeError(f"Unable to infer OHLC columns. Columns: {df.columns.tolist()}")
        else:
            raise RuntimeError(f"Missing OHLCV columns. Available columns: {df.columns.tolist()}")

    for c in ['Open','High','Low','Close','Volume']:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    try:
        if df.index.tz is not None:
            df.index = df.index.tz_convert(None)
    except Exception:
        pass

    df = df.sort_index()
    df.head(3).to_csv(os.path.join(out_dir,"yf_raw_head.csv"))
    df.tail(3).to_csv(os.path.join(out_dir,"yf_raw_tail.csv"))
    print(f"[fetch] rows: {len(df)}  from {df.index.min()} to {df.index.max()}  columns: {df.columns.tolist()}")
    return df

# ---------------- Improved EPS extractor ----------------
def _series_is_eps_like(name):
    n = name.lower()
    return ('eps' in n) or ('earnings per share' in n) or (re.search(r'\b(eps|earnings)\b', n) is not None)

def _index_row_matches_net_income(idx_name):
    n = str(idx_name).lower()
    return ('net' in n and 'income' in n) or ('netincome' in n) or ('profit' in n and 'loss' in n) or ('net loss' in n)

def _index_row_matches_shares(idx_name):
    n = str(idx_name).lower()
    return ('weighted' in n and 'share' in n) or ('shares' in n and ('outstanding' in n or 'basic' in n or 'weighted' in n)) or ('common shares' in n) or ('shareholder' in n and 'equity' not in n and 'shares' in n)

def fetch_quarterly_eps_yf_improved(symbol):
    """
    Robust best-effort EPS fetch using yfinance internals:
    returns DataFrame with columns ['fiscalDateEnding','eps'] (most recent first)
    """
    print("[earnings] attempting robust EPS extraction via yfinance...")
    t = yf.Ticker(symbol)
    candidates = []
    tried = []
    out = None

    attr_names = [
        "quarterly_earnings",
        "earnings",
        "quarterly_financials",
        "quarterly_income_stmt",
        "quarterly_cashflow",
        "financials",
        "income_stmt",
    ]

    for attr in attr_names:
        try:
            tried.append(attr)
            q = getattr(t, attr, None)
            if q is None:
                continue
            if isinstance(q, pd.DataFrame) and not q.empty:
                candidates.append((attr, q.copy()))
        except Exception:
            pass

    try:
        tried.append("get_earnings_history")
        hist = None
        if hasattr(t, "get_earnings_history"):
            hist = t.get_earnings_history()
        if hist:
            rows = []
            for h in hist:
                dt = h.get('startdatetime') or h.get('startdatetime')
                eps = h.get('epsactual') or h.get('epsestimate') or h.get('eps')
                try:
                    epsf = float(eps) if eps is not None else np.nan
                except Exception:
                    epsf = np.nan
                rows.append({'fiscalDateEnding': pd.to_datetime(dt, errors='coerce'), 'eps': epsf})
            if rows:
                df_hist = pd.DataFrame(rows).dropna(subset=['fiscalDateEnding']).sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)
                df_hist.to_csv(os.path.join(OUT_DIR, "yf_quarterly_eps_from_history.csv"), index=False)
                print("[earnings] extracted EPS from get_earnings_history()")
                return df_hist
    except Exception:
        pass

    for (attr, q) in candidates:
        try:
            q2 = q.copy()
            if all(isinstance(c, (pd.Timestamp, datetime)) or (isinstance(c, str) and re.match(r'\d{4}-\d{2}-\d{2}', c)) for c in q2.columns):
                idx_list = list(q2.index.astype(str))
                eps_row = None
                for r in idx_list:
                    if _series_is_eps_like(r):
                        eps_row = r
                        break
                if eps_row is not None:
                    s = q2.loc[eps_row].copy()
                    ser = pd.to_numeric(s.astype(str).str.replace(',','').replace('', np.nan), errors='coerce')
                    df_out = pd.DataFrame({'fiscalDateEnding': pd.to_datetime(ser.index.astype(str), errors='coerce'), 'eps': ser.values})
                    df_out = df_out.dropna(subset=['fiscalDateEnding']).sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)
                    df_out.to_csv(os.path.join(OUT_DIR, f"yf_eps_explicit_{attr}.csv"), index=False)
                    print(f"[earnings] found explicit EPS-like row '{eps_row}' in {attr}")
                    return df_out

                net_row = None
                shares_row = None
                for r in idx_list:
                    if net_row is None and _index_row_matches_net_income(r):
                        net_row = r
                    if shares_row is None and _index_row_matches_shares(r):
                        shares_row = r
                if net_row is not None and shares_row is not None:
                    net_ser = pd.to_numeric(q2.loc[net_row].astype(str).str.replace(',',''), errors='coerce')
                    shares_ser = pd.to_numeric(q2.loc[shares_row].astype(str).str.replace(',',''), errors='coerce')
                    cols = net_ser.index.intersection(shares_ser.index)
                    if len(cols) >= 1:
                        eps_values = []
                        dates = []
                        for c in cols:
                            ni = net_ser.loc[c]
                            sh = shares_ser.loc[c]
                            try:
                                epsv = float(ni) / float(sh) if (pd.notna(ni) and pd.notna(sh) and sh != 0) else np.nan
                            except Exception:
                                epsv = np.nan
                            eps_values.append(epsv)
                            try:
                                dates.append(pd.to_datetime(c, errors='coerce'))
                            except Exception:
                                dates.append(pd.NaT)
                        df_out = pd.DataFrame({'fiscalDateEnding': dates, 'eps': eps_values})
                        df_out = df_out.dropna(subset=['fiscalDateEnding']).reset_index(drop=True).sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)
                        df_out.to_csv(os.path.join(OUT_DIR, f"yf_eps_computed_net_shares_{attr}.csv"), index=False)
                        print(f"[earnings] computed EPS = NetIncome / Shares from rows '{net_row}' and '{shares_row}' in {attr}")
                        return df_out

            if all(isinstance(i, (pd.Timestamp, datetime)) or (isinstance(i, str) and re.match(r'\d{4}-\d{2}-\d{2}', i)) for i in q2.index.astype(str)):
                qt = q2.T
                idx_list = list(qt.index.astype(str))
                eps_col = None
                for c in qt.columns:
                    if _series_is_eps_like(str(c)):
                        eps_col = c
                        break
                if eps_col is not None:
                    ser = pd.to_numeric(qt[eps_col].astype(str).str.replace(',',''), errors='coerce')
                    df_out = pd.DataFrame({'fiscalDateEnding': pd.to_datetime(ser.index.astype(str), errors='coerce'), 'eps': ser.values})
                    df_out = df_out.dropna(subset=['fiscalDateEnding']).sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)
                    df_out.to_csv(os.path.join(OUT_DIR, f"yf_eps_explicit_transposed_{attr}.csv"), index=False)
                    print(f"[earnings] found explicit EPS-like column '{eps_col}' after transposing {attr}")
                    return df_out

                net_col = None
                shares_col = None
                for c in qt.columns:
                    if net_col is None and _index_row_matches_net_income(c):
                        net_col = c
                    if shares_col is None and _index_row_matches_shares(c):
                        shares_col = c
                if net_col is not None and shares_col is not None:
                    net_ser = pd.to_numeric(qt[net_col].astype(str).str.replace(',',''), errors='coerce')
                    shares_ser = pd.to_numeric(qt[shares_col].astype(str).str.replace(',',''), errors='coerce')
                    cols = net_ser.index.intersection(shares_ser.index)
                    if len(cols) >= 1:
                        eps_values = []
                        dates = []
                        for c in cols:
                            ni = net_ser.loc[c]
                            sh = shares_ser.loc[c]
                            try:
                                epsv = float(ni) / float(sh) if (pd.notna(ni) and pd.notna(sh) and sh != 0) else np.nan
                            except Exception:
                                epsv = np.nan
                            eps_values.append(epsv)
                            try:
                                dates.append(pd.to_datetime(c, errors='coerce'))
                            except Exception:
                                dates.append(pd.NaT)
                        df_out = pd.DataFrame({'fiscalDateEnding': dates, 'eps': eps_values})
                        df_out = df_out.dropna(subset=['fiscalDateEnding']).reset_index(drop=True).sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)
                        df_out.to_csv(os.path.join(OUT_DIR, f"yf_eps_computed_net_shares_transposed_{attr}.csv"), index=False)
                        print(f"[earnings] computed EPS from net/shares after transposing {attr}")
                        return df_out

        except Exception as e:
            print(f"[earnings] candidate {attr} parse error: {e}")
            continue

    print(f"[earnings] tried attributes: {tried}. No EPS-like data found. Returning empty eps DataFrame.")
    empty = pd.DataFrame(columns=['fiscalDateEnding','eps'])
    empty.to_csv(os.path.join(OUT_DIR, "yf_quarterly_eps_empty.csv"), index=False)
    return empty

# ---------------- Indicators ----------------
def compute_indicators(df):
    df = df.copy()
    df['EMA200'] = df['Close'].ewm(span=EMA_LEN, adjust=False).mean()
    ema_fast = df['Close'].ewm(span=MACD_FAST, adjust=False).mean()
    ema_slow = df['Close'].ewm(span=MACD_SLOW, adjust=False).mean()
    df['MACD'] = ema_fast - ema_slow
    df['MACD_SIGNAL'] = df['MACD'].ewm(span=MACD_SIGNAL, adjust=False).mean()

    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = (df['High'] - df['Close'].shift(1)).abs()
    df['L-PC'] = (df['Low'] - df['Close'].shift(1)).abs()
    df['TR'] = df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(ATR_LEN).mean()

    df['bull'] = (df['Close'] > df['Open'])
    df.drop(columns=['H-L','H-PC','L-PC','TR'], inplace=True, errors='ignore')

    # RSI(14)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0.0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0.0)).rolling(14).mean()
    rs = gain / (loss.replace(0, np.nan))
    df['RSI14'] = 100.0 - (100.0 / (1.0 + rs))
    df['RSI14'] = pd.to_numeric(df['RSI14'], errors='coerce')

    for c in ['EMA200','MACD','MACD_SIGNAL','ATR']:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    df[['Close','EMA200','MACD','MACD_SIGNAL','ATR','bull','RSI14']].head(3).to_csv(os.path.join(OUT_DIR,"ind_head.csv"))
    df[['Close','EMA200','MACD','MACD_SIGNAL','ATR','bull','RSI14']].tail(3).to_csv(os.path.join(OUT_DIR,"ind_tail.csv"))
    return df

def _is_missing(x):
    if is_scalar(x):
        return pd.isna(x)
    try:
        return bool(x.isna().any())
    except Exception:
        try:
            return np.isnan(x).any()
        except Exception:
            return False

# ---------------- Backtest engine implementing S1..S4 (S2 uses improved EPS) ----------------
def backtest(df, earnings_df=None):
    cash = INITIAL_CASH
    position = 0.0
    trade_log = []
    equity_curve = []

    # Enforced: no negative cash. We will count skipped events (when a debit would've made cash negative)
    skipped_count = 0

    # Small helper to attempt debit (returns True if succeeded, False if skipped)
    def attempt_debit(amount):
        nonlocal cash, skipped_count
        # round tolerance
        if amount <= (cash + 1e-9):
            cash -= amount
            return True
        else:
            # skip this operation due to insufficient cash
            skipped_count += 1
            return False

    # Small helper to attempt partial debit when possible (returns amount actually debited)
    def attempt_partial_debit(max_amount):
        nonlocal cash, skipped_count
        if cash <= 1e-12:
            skipped_count += 1
            return 0.0
        take = min(max_amount, cash)
        cash -= take
        # if partial (take < max_amount) count as a skipped event for the remainder
        if take + 1e-9 < max_amount:
            skipped_count += 1
        return take

    # S1 state
    s1_active = False
    s1_initial_amount = 0.0
    s1_stacks_used = 0
    s1_prev_stack_amount = None
    s1_prev_stack_price = None
    s1_initial_entry_price = None

    # S2 state
    s2_mode = None
    s2_active = False
    s2_is_short = False
    s2_tp = None

    # S3 state (Aggressive / Scalp C)
    s3_active = False
    s3_entry_price = None
    s3_entry_idx = None
    s3_tp = None
    s3_stop = None

    # S4
    s4_lots = []
    s4_last_entry_idx = -9999

    lots = []

    # determine S2 mode from earnings_df (Case A/B/C)
    try:
        if earnings_df is not None and 'eps' in earnings_df.columns and len(earnings_df.dropna(subset=['eps'])) >= 2:
            ed = earnings_df.dropna(subset=['eps']).copy().reset_index(drop=True)
            latest_eps = float(ed['eps'].iloc[0])
            prev_eps = float(ed['eps'].iloc[1])
            if prev_eps == 0:
                s2_mode = 'case_c'
                print("[S2] prev_eps was zero -> defaulting to CASE_C (technical)")
            else:
                eps_change = (latest_eps - prev_eps) / abs(prev_eps)
                if eps_change > EPS_CHANGE_LONG_THRESH:
                    s2_mode = 'long_only'; print(f"[S2] EPS change {eps_change:.2%} (> {EPS_CHANGE_LONG_THRESH:.0%}) -> LONG_ONLY (Case A)")
                elif eps_change < EPS_CHANGE_SHORT_THRESH:
                    s2_mode = 'short_only'; print(f"[S2] EPS change {eps_change:.2%} (< {EPS_CHANGE_SHORT_THRESH:.0%}) -> SHORT_ONLY (Case B)")
                else:
                    s2_mode = 'case_c'; print(f"[S2] EPS change {eps_change:.2%} ({EPS_CHANGE_SHORT_THRESH:.0%}-{EPS_CHANGE_LONG_THRESH:.0%}) -> CASE_C (use STEP-2 technical)")
        else:
            s2_mode = 'case_c'
            print("[S2] No/insufficient EPS values found -> default CASE_C (technical STEP-2)")
    except Exception as e:
        print("[S2] Error evaluating EPS change:", e)
        s2_mode = 'case_c'

    n = len(df)
    for idx in range(n):
        t = df.index[idx]
        row = df.iloc[idx]
        price = float(row['Close'])
        ema200 = row['EMA200']
        atr = row['ATR']
        macd = row['MACD']
        macd_sig = row['MACD_SIGNAL']

        if _is_missing(ema200) or _is_missing(atr) or _is_missing(macd):
            equity_curve.append({'time': t, 'equity': cash + position * price})
            continue

        ema200 = float(ema200)
        atr = float(atr)
        macd = float(macd)
        macd_sig = float(macd_sig) if not _is_missing(macd_sig) else float('nan')

        ema_down = False
        if idx >= 1:
            ema_down = float(df['EMA200'].iloc[idx]) < float(df['EMA200'].iloc[idx-1])
        ema_up = not ema_down

        # ---------- S1 ----------
        if (price < ema200) and ema_down and (not s1_active):
            buy_amount = cash * S1_INIT_PCT
            # compute cost and confirm cash is available via attempt_debit
            if buy_amount >= 0.5:
                # attempt to debit the exact buy_amount (prevents negative cash)
                if attempt_debit(buy_amount):
                    shares = buy_amount / price
                    position += shares
                    s1_active = True
                    s1_initial_amount = buy_amount
                    s1_stacks_used = 0
                    s1_prev_stack_amount = None
                    s1_prev_stack_price = price
                    s1_initial_entry_price = price
                    lots.append({'strategy':'S1','side':'long','shares':shares,'entry_price':price,'entry_time':t,'entry_reason':'S1_INIT'})
                    trade_log.append({'time':t,'strategy':'S1','type':'S1_INIT_BUY','price':price,'shares':shares,'cash':cash,'reason':'S1_INIT'})
                else:
                    # skipped due to insufficient cash
                    pass

        if s1_active and s1_stacks_used < S1_MAX_STACKS and idx >= 1:
            prev_close = float(df['Close'].iloc[idx-1])
            if s1_prev_stack_price is not None:
                required_level = s1_prev_stack_price * (1.0 - 0.02)
            else:
                required_level = (s1_initial_entry_price if s1_initial_entry_price is not None else prev_close) * (1.0 - 0.02)
            base_add_amount = s1_initial_amount * S1_STACK_REL_TO_INITIAL
            add_amount = base_add_amount * (S1_STACK_DECAY ** s1_stacks_used)
            # don't min with cash here; we'll attempt debit
            price_condition = (price <= required_level)
            ema_condition = price < ema200
            allow_stack = False
            if price_condition and ema_condition:
                if s1_prev_stack_amount is None or (add_amount < s1_prev_stack_amount - 1e-12):
                    allow_stack = True
            if allow_stack and add_amount >= 0.5:
                # attempt to debit the add_amount; if insufficient cash, skip and increment skipped_count (handled in attempt_debit)
                if attempt_debit(add_amount):
                    shares = add_amount / price
                    position += shares
                    s1_stacks_used += 1
                    trade_log.append({'time':t,'strategy':'S1','type':'S1_STACK_BUY','price':price,'shares':shares,'stack_num':s1_stacks_used,'cash':cash,'reason':f'S1_STACK_{s1_stacks_used}'})
                    lots.append({'strategy':'S1','side':'long','shares':shares,'entry_price':price,'entry_time':t,'entry_reason':f'S1_STACK_{s1_stacks_used}','stack_num':s1_stacks_used})
                    s1_prev_stack_amount = add_amount
                    s1_prev_stack_price = price
                else:
                    # skipped due to insufficient cash
                    pass

        if s1_active:
            s1_lots = [l for l in lots if l['strategy']=='S1' and l['side']=='long']
            avg_entry_price = None
            if s1_lots:
                total_shares = sum([l['shares'] for l in s1_lots])
                if total_shares > 0:
                    weighted = sum([l['shares'] * l['entry_price'] for l in s1_lots])
                    avg_entry_price = weighted / total_shares
            required_exit_price = ema200 * 1.049
            if USE_OPTION_C and (avg_entry_price is not None):
                profit_level = avg_entry_price * S1_OPTION_C_PROFIT_PCT
                required_exit_price = max(required_exit_price, profit_level)
            if price >= required_exit_price:
                s1_shares = sum([l['shares'] for l in lots if l['strategy']=='S1' and l['side']=='long'])
                if s1_shares > 0:
                    # selling long: this is a credit so always allowed
                    cash += s1_shares * price
                    position -= s1_shares
                    trade_log.append({'time':t,'strategy':'S1','type':'S1_EXIT_4p9pct','price':price,'shares':-s1_shares,'cash':cash,'reason':'S1_EXIT_4p9pct'})
                    lots = [l for l in lots if not (l['strategy']=='S1' and l['side']=='long')]
                s1_active = False
                s1_initial_amount = 0.0
                s1_stacks_used = 0
                s1_prev_stack_amount = None
                s1_prev_stack_price = None
                s1_initial_entry_price = None

        # ---------------- S2 (uses s2_mode detected from EPS) ----------------
        if not s2_active:
            nav = cash + position * price
            notional = nav * S2_PCT
            if notional >= 0.5:
                if s2_mode == 'long_only':
                    if ema_up:
                        # attempt to debit notional cost
                        cost = notional
                        if attempt_debit(cost):
                            shares = cost / price
                            position += shares
                            s2_active = True
                            s2_is_short = False
                            s2_tp = price + S2_TP_RR * atr
                            trade_log.append({'time':t,'strategy':'S2','type':'LONG_INIT','price':price,'shares':shares,'cash':cash,'reason':'S2_LONG_A_eps>20pct','tp':s2_tp})
                            lots.append({'strategy':'S2','side':'long','shares':shares,'entry_price':price,'entry_time':t,'entry_reason':'S2_LONG_A'})
                        else:
                            # skipped due to insufficient cash
                            pass
                elif s2_mode == 'short_only':
                    if ema_down:
                        # short entry receives cash; allowed
                        shares = notional / price
                        position -= shares
                        cash += shares * price
                        s2_active = True
                        s2_is_short = True
                        s2_tp = price - S2_TP_RR * atr
                        trade_log.append({'time':t,'strategy':'S2','type':'SHORT_INIT','price':price,'shares':-shares,'cash':cash,'reason':'S2_SHORT_B_eps<-20pct','tp':s2_tp})
                        lots.append({'strategy':'S2','side':'short','shares':shares,'entry_price':price,'entry_time':t,'entry_reason':'S2_SHORT_B'})
                elif s2_mode == 'case_c':
                    if (price > ema200 * 1.019) and ema_up:
                        cost = notional
                        if attempt_debit(cost):
                            shares = cost / price
                            position += shares
                            s2_active = True
                            s2_is_short = False
                            s2_tp = price + S2_TP_RR * atr
                            trade_log.append({'time':t,'strategy':'S2','type':'LONG_INIT','price':price,'shares':shares,'cash':cash,'reason':'S2_LONG_C_STEP2','tp':s2_tp})
                            lots.append({'strategy':'S2','side':'long','shares':shares,'entry_price':price,'entry_time':t,'entry_reason':'S2_LONG_C'})
                        else:
                            # skipped due to insufficient cash
                            pass
                    elif (price < ema200 * 1.019) and ema_down:
                        # short entry receives cash; allowed
                        shares = notional / price
                        position -= shares
                        cash += shares * price
                        s2_active = True
                        s2_is_short = True
                        s2_tp = price - S2_TP_RR * atr
                        trade_log.append({'time':t,'strategy':'S2','type':'SHORT_INIT','price':price,'shares':-shares,'cash':cash,'reason':'S2_SHORT_C_STEP2','tp':s2_tp})
                        lots.append({'strategy':'S2','side':'short','shares':shares,'entry_price':price,'entry_time':t,'entry_reason':'S2_SHORT_C'})

        if s2_active:
            if s2_is_short and position < 0:
                if price <= s2_tp:
                    cover_shares = sum([l['shares'] for l in lots if l['strategy']=='S2' and l['side']=='short'])
                    if cover_shares > 0:
                        # covering a short costs cash. allow partial cover if not enough cash
                        total_cost = cover_shares * price
                        if cash + 1e-9 >= total_cost:
                            # full cover
                            cash -= total_cost
                            position += cover_shares
                            trade_log.append({'time':t,'strategy':'S2','type':'SHORT_TP','price':price,'shares':cover_shares,'cash':cash,'reason':'S2_SHORT_TP'})
                            lots = [l for l in lots if not (l['strategy']=='S2' and l['side']=='short')]
                        else:
                            # partial cover allowed: buy as many shares as cash allows
                            affordable = math.floor((cash / price) * 1e9) / 1e9  # avoid floating issues; keep many decimals
                            if affordable > 0:
                                actual_cost = affordable * price
                                cash -= actual_cost
                                position += affordable
                                trade_log.append({'time':t,'strategy':'S2','type':'SHORT_TP_PARTIAL','price':price,'shares':affordable,'cash':cash,'reason':'S2_SHORT_TP_PARTIAL'})
                                # remove/adjust lots proportionally
                                remaining = affordable
                                new_lots = []
                                for l in lots:
                                    if not (l['strategy']=='S2' and l['side']=='short'):
                                        new_lots.append(l)
                                        continue
                                    if remaining <= 0:
                                        new_lots.append(l)
                                    else:
                                        if l['shares'] <= remaining + 1e-12:
                                            remaining -= l['shares']
                                            # this lot fully consumed; do not append
                                        else:
                                            l['shares'] -= remaining
                                            new_lots.append(l)
                                            remaining = 0
                                lots = new_lots
                                skipped_count += 1  # because full cover couldn't be done
                            else:
                                skipped_count += 1
                    s2_active = False
                    s2_is_short = False
                    s2_tp = None
            elif (not s2_is_short) and position > 0:
                if price >= s2_tp:
                    sell_shares = sum([l['shares'] for l in lots if l['strategy']=='S2' and l['side']=='long'])
                    if sell_shares > 0:
                        # selling long is a credit
                        cash += sell_shares * price
                        position -= sell_shares
                        trade_log.append({'time':t,'strategy':'S2','type':'LONG_TP','price':price,'shares':-sell_shares,'cash':cash,'reason':'S2_LONG_TP'})
                        lots = [l for l in lots if not (l['strategy']=='S2' and l['side']=='long')]
                    s2_active = False
                    s2_is_short = False
                    s2_tp = None

        # ---------------- S3 (Aggressive / Scalp C) - ENTRY ----------------
        if (price < ema200) and ema_down and (not s3_active):
            rsi_val = df['RSI14'].iloc[idx] if 'RSI14' in df.columns else None
            if (rsi_val is not None) and (not _is_missing(rsi_val)) and (rsi_val > 55.0):
                nav = cash + position * price
                notional = nav * S3_PCT   # 7% NAV
                if notional >= 0.5:
                    # S3 is a short (in your code). Short entry gives cash (we credit)
                    shares = notional / price
                    position -= shares
                    cash += shares * price
                    s3_active = True
                    s3_entry_price = price
                    s3_entry_idx = idx
                    s3_tp = s3_entry_price - S3_TP_RR * atr
                    s3_stop = s3_entry_price + S3_STOP_MULT * atr  # S3_STOP_MULT = 0.4
                    lots.append({'strategy':'S3','side':'short','shares':shares,'entry_price':price,'entry_time':t,'entry_idx':idx,'entry_reason':'S3_SCALP_C_RSI'})
                    trade_log.append({'time':t,'strategy':'S3','type':'SHORT_INIT','price':price,'shares':-shares,'cash':cash,'reason':'S3_SHORT_INIT_SCALP_C','tp':s3_tp,'stop':s3_stop,'entry_idx':idx})

        # ---------------- S3 (Aggressive / Scalp C) - EXIT ----------------
        if s3_active and position < 0:
            # 1) TP hit (full cover)
            if price <= s3_tp:
                cover_shares = sum([l['shares'] for l in lots if l['strategy']=='S3' and l['side']=='short'])
                if cover_shares > 0:
                    total_cost = cover_shares * price
                    # attempt to debit cover cost; allow partial cover if necessary
                    if cash + 1e-9 >= total_cost:
                        cash -= total_cost
                        position += cover_shares
                        trade_log.append({'time':t,'strategy':'S3','type':'SHORT_TP','price':price,'shares':cover_shares,'cash':cash,'reason':'S3_SHORT_TP_SCALP'})
                        lots = [l for l in lots if not (l['strategy']=='S3' and l['side']=='short')]
                    else:
                        affordable = math.floor((cash / price) * 1e9) / 1e9
                        if affordable > 0:
                            actual_cost = affordable * price
                            cash -= actual_cost
                            position += affordable
                            trade_log.append({'time':t,'strategy':'S3','type':'SHORT_TP_PARTIAL','price':price,'shares':affordable,'cash':cash,'reason':'S3_SHORT_TP_PARTIAL_SCALP'})
                            # adjust lots
                            remaining = affordable
                            new_lots = []
                            for l in lots:
                                if not (l['strategy']=='S3' and l['side']=='short'):
                                    new_lots.append(l)
                                    continue
                                if remaining <= 0:
                                    new_lots.append(l)
                                else:
                                    if l['shares'] <= remaining + 1e-12:
                                        remaining -= l['shares']
                                    else:
                                        l['shares'] -= remaining
                                        new_lots.append(l)
                                        remaining = 0
                            lots = new_lots
                            skipped_count += 1
                        else:
                            skipped_count += 1
                s3_active=False; s3_entry_price=None; s3_entry_idx=None; s3_tp=None; s3_stop=None

            # 2) Stop hit (tight stop)
            elif price >= s3_stop:
                cover_shares = sum([l['shares'] for l in lots if l['strategy']=='S3' and l['side']=='short'])
                if cover_shares > 0:
                    total_cost = cover_shares * price
                    if cash + 1e-9 >= total_cost:
                        cash -= total_cost
                        position += cover_shares
                        trade_log.append({'time':t,'strategy':'S3','type':'SHORT_STOP','price':price,'shares':cover_shares,'cash':cash,'reason':'S3_SHORT_STOP_SCALP'})
                        lots = [l for l in lots if not (l['strategy']=='S3' and l['side']=='short')]
                    else:
                        affordable = math.floor((cash / price) * 1e9) / 1e9
                        if affordable > 0:
                            actual_cost = affordable * price
                            cash -= actual_cost
                            position += affordable
                            trade_log.append({'time':t,'strategy':'S3','type':'SHORT_STOP_PARTIAL','price':price,'shares':affordable,'cash':cash,'reason':'S3_SHORT_STOP_PARTIAL_SCALP'})
                            # adjust lots
                            remaining = affordable
                            new_lots = []
                            for l in lots:
                                if not (l['strategy']=='S3' and l['side']=='short'):
                                    new_lots.append(l)
                                    continue
                                if remaining <= 0:
                                    new_lots.append(l)
                                else:
                                    if l['shares'] <= remaining + 1e-12:
                                        remaining -= l['shares']
                                    else:
                                        l['shares'] -= remaining
                                        new_lots.append(l)
                                        remaining = 0
                            lots = new_lots
                            skipped_count += 1
                        else:
                            skipped_count += 1
                s3_active=False; s3_entry_price=None; s3_entry_idx=None; s3_tp=None; s3_stop=None

            else:
                # 3) EMA flip early exit (failure)
                if price > ema200:
                    cover_shares = sum([l['shares'] for l in lots if l['strategy']=='S3' and l['side']=='short'])
                    if cover_shares > 0:
                        total_cost = cover_shares * price
                        if cash + 1e-9 >= total_cost:
                            cash -= total_cost
                            position += cover_shares
                            trade_log.append({'time':t,'strategy':'S3','type':'SHORT_EARLY_EXIT_EMA','price':price,'shares':cover_shares,'cash':cash,'reason':'S3_EARLY_EXIT_EMA_SCALP'})
                            lots = [l for l in lots if not (l['strategy']=='S3' and l['side']=='short')]
                        else:
                            affordable = math.floor((cash / price) * 1e9) / 1e9
                            if affordable > 0:
                                actual_cost = affordable * price
                                cash -= actual_cost
                                position += affordable
                                trade_log.append({'time':t,'strategy':'S3','type':'SHORT_EARLY_EXIT_EMA_PARTIAL','price':price,'shares':affordable,'cash':cash,'reason':'S3_EARLY_EXIT_EMA_PARTIAL_SCALP'})
                                # adjust lots
                                remaining = affordable
                                new_lots = []
                                for l in lots:
                                    if not (l['strategy']=='S3' and l['side']=='short'):
                                        new_lots.append(l)
                                        continue
                                    if remaining <= 0:
                                        new_lots.append(l)
                                    else:
                                        if l['shares'] <= remaining + 1e-12:
                                            remaining -= l['shares']
                                        else:
                                            l['shares'] -= remaining
                                            new_lots.append(l)
                                            remaining = 0
                                lots = new_lots
                                skipped_count += 1
                            else:
                                skipped_count += 1
                    s3_active=False; s3_entry_price=None; s3_entry_idx=None; s3_tp=None; s3_stop=None

                # 4) Max-hold bars safety
                elif (s3_entry_idx is not None) and ((idx - s3_entry_idx) >= S3_MAX_HOLD_BARS):
                    cover_shares = sum([l['shares'] for l in lots if l['strategy']=='S3' and l['side']=='short'])
                    if cover_shares > 0:
                        total_cost = cover_shares * price
                        if cash + 1e-9 >= total_cost:
                            cash -= total_cost
                            position += cover_shares
                            trade_log.append({'time':t,'strategy':'S3','type':'SHORT_EARLY_EXIT_MAX_HOLD','price':price,'shares':cover_shares,'cash':cash,'reason':'S3_EARLY_EXIT_MAX_HOLD_SCALP','held_bars': idx - s3_entry_idx})
                            lots = [l for l in lots if not (l['strategy']=='S3' and l['side']=='short')]
                        else:
                            affordable = math.floor((cash / price) * 1e9) / 1e9
                            if affordable > 0:
                                actual_cost = affordable * price
                                cash -= actual_cost
                                position += affordable
                                trade_log.append({'time':t,'strategy':'S3','type':'SHORT_EARLY_EXIT_MAX_HOLD_PARTIAL','price':price,'shares':affordable,'cash':cash,'reason':'S3_EARLY_EXIT_MAX_HOLD_PARTIAL_SCALP','held_bars': idx - s3_entry_idx})
                                # adjust lots
                                remaining = affordable
                                new_lots = []
                                for l in lots:
                                    if not (l['strategy']=='S3' and l['side']=='short'):
                                        new_lots.append(l)
                                        continue
                                    if remaining <= 0:
                                        new_lots.append(l)
                                    else:
                                        if l['shares'] <= remaining + 1e-12:
                                            remaining -= l['shares']
                                        else:
                                            l['shares'] -= remaining
                                            new_lots.append(l)
                                            remaining = 0
                                lots = new_lots
                                skipped_count += 1
                            else:
                                skipped_count += 1
                    s3_active=False; s3_entry_price=None; s3_entry_idx=None; s3_tp=None; s3_stop=None

        # ---------------- S4 (unchanged safe default) ----------------
        macd_cross_up = (macd > macd_sig) and (df['MACD'].shift(1).iloc[idx] <= df['MACD_SIGNAL'].shift(1).iloc[idx])
        if macd_cross_up and macd > 0 and price > ema200:
            nav = cash + position * price
            current_s4_notional = sum([lot['entry_price'] * lot['shares'] for lot in s4_lots])
            max_allowed_s4_notional = nav * S4_MAX_CONCURRENT_PCT
            n_open_s4 = len(s4_lots)
            bars_since_last_s4 = idx - s4_last_entry_idx
            desired_notional = nav * S4_PCT
            remaining_notional_capacity = max(0.0, max_allowed_s4_notional - current_s4_notional)
            can_open_more_lots = (n_open_s4 < S4_MAX_OPEN_LOTS)
            cooldown_ok = (bars_since_last_s4 >= S4_COOLDOWN_BARS)
            alloc_notional = 0.0
            if can_open_more_lots and cooldown_ok and remaining_notional_capacity > 0.0:
                alloc_notional = min(desired_notional, remaining_notional_capacity)
            if alloc_notional >= 0.5:
                # attempt debit for alloc_notional
                if attempt_debit(alloc_notional):
                    shares = alloc_notional / price
                    position += shares
                    entry_stop = price - S4_RR_STOP * atr
                    entry_tp = price + S4_RR_TP * atr
                    s4_lots.append({'shares':shares,'entry_price':price,'entry_time':t,'stop':entry_stop,'tp':entry_tp})
                    lots.append({'strategy':'S4','side':'long','shares':shares,'entry_price':price,'entry_time':t,'entry_reason':'S4_MACD_INIT','stop':entry_stop,'tp':entry_tp})
                    trade_log.append({'time':t,'strategy':'S4','type':'S4_MACD_BUY','price':price,'shares':shares,'cash':cash,'reason':'S4_MACD_INIT','stop':entry_stop,'tp':entry_tp})
                    s4_last_entry_idx = idx
                else:
                    # skipped due to insufficient cash
                    pass

        if s4_lots:
            remaining_lots = []
            for lot in s4_lots:
                if price >= lot['tp']:
                    # take profit -> credit
                    cash += lot['shares'] * price
                    position -= lot['shares']
                    trade_log.append({'time':t,'strategy':'S4','type':'S4_TP','price':price,'shares':-lot['shares'],'cash':cash,'reason':'S4_TP'})
                    lots = [l for l in lots if not (l['strategy']=='S4' and abs(l['shares'] - lot['shares'])<1e-9 and l.get('entry_price')==lot['entry_price'] and l.get('entry_time')==lot['entry_time'])]
                elif price <= lot['stop']:
                    # stop -> credit (sell)
                    cash += lot['shares'] * price
                    position -= lot['shares']
                    trade_log.append({'time':t,'strategy':'S4','type':'S4_STOP','price':price,'shares':-lot['shares'],'cash':cash,'reason':'S4_STOP'})
                    lots = [l for l in lots if not (l['strategy']=='S4' and abs(l['shares'] - lot['shares'])<1e-9 and l.get('entry_price')==lot['entry_price'] and l.get('entry_time')==lot['entry_time'])]
                else:
                    remaining_lots.append(lot)
            s4_lots = remaining_lots

        # end-of-bar equity snapshot
        equity_curve.append({'time': t, 'equity': cash + position * price})

    # After main loop ends, append SKIPPED_SUMMARY event to trade_log (single aggregate entry)
    # This gives you visibility of how many events were skipped due to insufficient cash.
    if skipped_count > 0:
        # timestamp uses last bar time if available
        last_time = df.index[-1] if len(df) > 0 else datetime.utcnow()
        trade_log.append({'time': last_time, 'strategy':'SYSTEM', 'type':'SKIPPED_SUMMARY', 'skipped_count': skipped_count, 'reason':'skipped_aggregate'})

    trades_df = pd.DataFrame(trade_log)
    equity_df = pd.DataFrame(equity_curve).set_index('time')
    trades_df.to_csv(os.path.join(OUT_DIR,"trades_raw.csv"), index=False)
    equity_df.to_csv(os.path.join(OUT_DIR,"equity_raw.csv"))
    return trades_df, equity_df

# ---------------- RUN workflow ----------------
print(">>> FETCH PRICE")
df_raw = fetch_yf_intraday(SYMBOL, period=PERIOD, interval=INTERVAL, out_dir=OUT_DIR)

print(">>> COMPUTE INDICATORS")
df = compute_indicators(df_raw.copy())

print(">>> FETCH EPS (improved)")
earnings_df = fetch_quarterly_eps_yf_improved(SYMBOL)
print("== EARNINGS_DF DIAGNOSTICS ===")
print("earnings_df shape:", getattr(earnings_df,'shape',None))
print(earnings_df.head(10))
earnings_df.to_csv(os.path.join(OUT_DIR,"debug_earnings_df.csv"), index=False)
print("=== END DIAGNOSTICS ===")

print(f">>> Bars fetched: {len(df)}. NaNs in EMA200/ATR/MACD? {df['EMA200'].isna().sum()}, {df['ATR'].isna().sum()}, {df['MACD'].isna().sum()}")
if len(df) < MIN_BARS_REQUIRED:
    print(f"WARNING: only {len(df)} bars (<{MIN_BARS_REQUIRED}). EMA200 may be unreliable; consider larger PERIOD or daily timeframe.")

print(">>> RUN BACKTEST")
trades, equity = backtest(df, earnings_df=earnings_df)
print(f">>> Backtest done. Trades recorded: {len(trades)}, Equity points: {len(equity)}")

# ---------------- POST PROCESS (same as before) ----------------
if trades is None or len(trades) == 0:
    print("No trades were recorded by the backtest. Nothing to summarize.")
else:
    trades_df = trades.sort_values('time').reset_index(drop=True).copy()
    trades_df['shares'] = pd.to_numeric(trades_df['shares'], errors='coerce').fillna(0.0)
    trades_df['price'] = pd.to_numeric(trades_df['price'], errors='coerce').fillna(0.0)

    ledger = []
    prev_cash = INITIAL_CASH
    prev_pos = 0.0
    trade_id = 0

    for i, r in trades_df.iterrows():
        trade_id += 1
        t = r.get('time')
        strat = r.get('strategy', '')
        typ = r.get('type', '')
        reason = r.get('reason', '')
        # Some records, like SKIPPED_SUMMARY, might not have price/shares numeric
        shares = float(r['shares']) if pd.notna(r.get('shares')) else 0.0
        price = float(r['price']) if pd.notna(r.get('price')) else 0.0

        cash_before = prev_cash
        pos_before = prev_pos

        delta_cash = - shares * price
        cash_after = cash_before + delta_cash
        pos_after = pos_before + shares

        ledger.append({
            'trade_id': trade_id,
            'time': t,
            'strategy': strat,
            'type': typ,
            'price': price,
            'shares': shares,
            'cash_before': cash_before,
            'cash_after': cash_after,
            'pos_before': pos_before,
            'pos_after': pos_after,
            'reason': reason
        })

        prev_cash = cash_after
        prev_pos = pos_after

    ledger_df = pd.DataFrame(ledger)
    ledger_csv = os.path.join(OUT_DIR, "trades_full_verbose.csv")
    ledger_df.to_csv(ledger_csv, index=False)
    print(f"[OUTPUT] verbose ledger saved: {ledger_csv}")

    pd.set_option('display.max_rows', 20)
    pd.set_option('display.width', 160)
    print("\n--- Trades Full Verbose (first 8 rows) ---")
    print(ledger_df.head(8).to_string(index=False))
    print("\n--- Trades Full Verbose (last 8 rows) ---")
    print(ledger_df.tail(8).to_string(index=False))
    print(f"\nTotal trade events: {len(ledger_df)}")

    # ---------- Robust FIFO round-trip summary (per-strategy, per-side) ----------
    summary_rows = []

    # Maintain per-strategy FIFO lists of open lots for long and short
    open_longs = {}   # strategy -> list of {'shares', 'entry_price','entry_time','entry_reason','entry_value'}
    open_shorts = {}  # strategy -> list of {'shares', 'entry_price','entry_time','entry_reason','entry_value_received'}

    def push_long(strat, shares, price, time, reason):
        if strat not in open_longs:
            open_longs[strat] = []
        open_longs[strat].append({'shares': shares, 'entry_price': price, 'entry_time': time, 'entry_reason': reason, 'entry_value': shares * price})

    def push_short(strat, shares, price, time, reason):
        # For shorts we store the cash received at entry (shares * price)
        if strat not in open_shorts:
            open_shorts[strat] = []
        open_shorts[strat].append({'shares': shares, 'entry_price': price, 'entry_time': time, 'entry_reason': reason, 'entry_value_received': shares * price})

    def consume_long(strat, shares_to_sell, exit_price, exit_time, exit_reason):
        # FIFO consume long lots -> create round-trip rows
        remaining = shares_to_sell
        if strat not in open_longs:
            return  # nothing to match
        while remaining > 1e-12 and open_longs[strat]:
            lot = open_longs[strat][0]
            take = min(lot['shares'], remaining)
            entry_shares = take
            entry_price = lot['entry_price']
            entry_time = lot['entry_time']
            entry_reason = lot.get('entry_reason','')
            pnl = (exit_price * entry_shares) - (entry_price * entry_shares)
            summary_rows.append({
                'strategy': strat,
                'direction': 'LONG',
                'entry_time': entry_time,
                'entry_price': entry_price,
                'exit_time': exit_time,
                'exit_price': exit_price,
                'pnl': pnl,
                'entry_reason': entry_reason,
                'exit_reason': exit_reason,
                'shares': entry_shares
            })
            lot['shares'] -= take
            if lot['shares'] <= 1e-12:
                open_longs[strat].pop(0)
            remaining -= take

    def consume_short(strat, shares_to_cover, exit_price, exit_time, exit_reason):
        # FIFO consume short lots -> create round-trip rows
        remaining = shares_to_cover
        if strat not in open_shorts:
            return
        while remaining > 1e-12 and open_shorts[strat]:
            lot = open_shorts[strat][0]
            take = min(lot['shares'], remaining)
            entry_shares = take
            entry_price = lot['entry_price']  # price at short entry
            entry_time = lot['entry_time']
            entry_reason = lot.get('entry_reason','')
            entry_value_received = entry_shares * entry_price
            exit_cost = entry_shares * exit_price
            pnl = entry_value_received - exit_cost
            summary_rows.append({
                'strategy': strat,
                'direction': 'SHORT',
                'entry_time': entry_time,
                'entry_price': entry_price,
                'exit_time': exit_time,
                'exit_price': exit_price,
                'pnl': pnl,
                'entry_reason': entry_reason,
                'exit_reason': exit_reason,
                'shares': entry_shares
            })
            lot['shares'] -= take
            if lot['shares'] <= 1e-12:
                open_shorts[strat].pop(0)
            remaining -= take

    # Iterate ledger events in chronological order and build FIFO matches
    for i, r in ledger_df.iterrows():
        typ = (r['type'] or '').upper()
        strat = r.get('strategy','')
        shares = r['shares'] if pd.notna(r.get('shares')) else 0.0
        price = r['price'] if pd.notna(r.get('price')) else 0.0
        t = r['time']
        reason = r.get('reason', typ)

        # ENTRY LONG (positive shares that are buys)
        if shares > 0 and ('INIT' in typ or 'BUY' in typ or 'LONG_INIT' in typ or (strat and typ=='')):
            push_long(strat, shares, price, t, reason)
            continue

        # EXIT LONG (negative shares that reduce position)
        if shares < 0 and ('TP' in typ or 'EXIT' in typ or 'SELL' in typ or 'STOP' in typ):
            consume_long(strat, abs(shares), price, t, typ)
            continue

        # ENTRY SHORT (negative shares labelled SHORT_INIT or similar)
        if shares < 0 and ('SHORT_INIT' in typ or ('SHORT' in typ and 'INIT' in typ) or ('SHORT' in typ and 'INIT' not in typ and 'TP' not in typ and 'STOP' not in typ)):
            push_short(strat, abs(shares), price, t, reason)
            continue

        # COVER / EXIT SHORT (positive shares that close short)
        if shares > 0 and (('SHORT' in typ and ('TP' in typ or 'STOP' in typ or 'COVER' in typ)) or ('SHORT_PARTIAL' in typ) or ('SHORT_TP' in typ) or ('SHORT_STOP' in typ)):
            consume_short(strat, shares, price, t, typ)
            continue

        # Fallback heuristics
        if shares < 0 and 'SELL' in typ:
            consume_long(strat, abs(shares), price, t, typ)
            continue
        if shares > 0 and 'COVER' in typ:
            consume_short(strat, shares, price, t, typ)
            continue

    # Build DataFrame
    summary_df = pd.DataFrame(summary_rows)
    if not summary_df.empty:
        try:
            summary_df['entry_time'] = pd.to_datetime(summary_df['entry_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
            summary_df['exit_time'] = pd.to_datetime(summary_df['exit_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
        except Exception:
            pass
        summary_df = summary_df[['strategy','direction','shares','entry_time','entry_price','exit_time','exit_price','pnl','entry_reason','exit_reason']]
        summary_csv = os.path.join(OUT_DIR, "summary_trades.csv")
        summary_df.to_csv(summary_csv, index=False)
        print(f"[OUTPUT] round-trip summary saved: {summary_csv}")
        print("\n--- Summary trades (first 10) ---")
        print(summary_df.head(10).to_string(index=False))
    else:
        print("No round-trip summary trades found; summary_trades.csv not created.")

    # per-strategy pnl
    if not summary_df.empty:
        strat_stats = []
        for strat, g in summary_df.groupby('strategy'):
            g = g.copy()
            total_pnl = g['pnl'].sum()
            num_trades = len(g)
            wins = (g['pnl'] > 0).sum()
            losses = (g['pnl'] <= 0).sum()
            avg_pnl = g['pnl'].mean()
            strat_stats.append({'strategy': strat, 'num_trades': num_trades, 'net_pnl': total_pnl, 'avg_pnl': avg_pnl, 'wins': wins, 'losses': losses})
        strat_stats_df = pd.DataFrame(strat_stats).sort_values('net_pnl', ascending=False)
        strat_stats_csv = os.path.join(OUT_DIR, "strategy_pnl.csv")
        strat_stats_df.to_csv(strat_stats_csv, index=False)
        print(f"[OUTPUT] per-strategy PnL saved: {strat_stats_csv}")
        print("\n--- Per-strategy PnL ---")
        print(strat_stats_df.to_string(index=False))
    else:
        print("No strategy-level summary available (no round-trip trades).")

# Equity stats and plots
if equity is not None and len(equity) > 0:
    eq = equity.copy()
    eq.sort_index(inplace=True)
    eq['equity'] = eq['equity'].astype(float)
    eq['peak'] = eq['equity'].cummax()
    eq['drawdown'] = eq['equity'] / eq['peak'] - 1.0
    max_dd = float(eq['drawdown'].min())
    start_equity = float(eq['equity'].iloc[0])
    end_equity = float(eq['equity'].iloc[-1])
    ret_pct = (end_equity / start_equity - 1.0) * 100.0
    print("\n=== Equity Stats ===")
    print(f"Start equity: {start_equity:,.2f}")
    print(f"End equity:   {end_equity:,.2f}")
    print(f"Return:       {ret_pct:.2f}%")
    print(f"Max Drawdown: {max_dd*100:.2f}%")

    equity_csv = os.path.join(OUT_DIR, "equity_curve.csv")
    eq[['equity','peak','drawdown']].to_csv(equity_csv)
    print(f"[OUTPUT] equity curve saved: {equity_csv}")

    try:
        plt.figure(figsize=(10,4))
        plt.plot(eq.index, eq['equity'])
        plt.title("Equity Curve")
        plt.grid(True)
        plt.tight_layout()
        eq_plot = os.path.join(OUT_DIR, "equity_curve.png")
        plt.savefig(eq_plot)
        plt.close()
        print(f"[OUTPUT] saved plot: {eq_plot}")

        plt.figure(figsize=(10,3))
        plt.plot(eq.index, eq['drawdown'])
        plt.title("Drawdown")
        plt.grid(True)
        plt.tight_layout()
        dd_plot = os.path.join(OUT_DIR, "drawdown.png")
        plt.savefig(dd_plot)
        plt.close()
        print(f"[OUTPUT] saved plot: {dd_plot}")
    except Exception as e:
        print("Plotting failed:", e)
else:
    print("No equity series available to compute stats/plots.")

print("\nAll outputs in folder:", OUT_DIR)
print("Files (key): trades_full_verbose.csv, trades_raw.csv, summary_trades.csv, strategy_pnl.csv, equity_curve.csv, equity_curve.png, drawdown.png")